# 第1部 基礎編: データ加工

## 第1章 ウェブからの注文数を分析する10本ノック

### ノック1: データを読み込んでみよう

ここでは、ある企業のECサイトでの商品の注文数の推移を分析していきます。  
データの属性を理解し、分析をするためにデータを加工した後、  
データの可視化を行うことで問題を発見していくプロセスを学びます。

In [ ]:
import polars as pl

In [ ]:
# 列表示を省略しない
pl.Config.set_tbl_cols(-1)
# 行の表示を省略しない
pl.Config.set_tbl_rows(-1)

#### 顧客マスター

| 列名 | データ例 | データ型 | 列の説明 |
| ---- | ---- | ---- | ---- |
| customer_id        | IK152942                    | str | 顧客ID |
| customer_name      | 平田 裕次郎                 | str | 顧客名 |
| registration_date  | 2019-01-01 00:25:33         | str | 登録日時 |
| customer_name_kana | ひらた ゆうじろう           | str | 顧客名(ひらがな) |
| email              | hirata_yuujirou@example.com | str | メールアドレス |
| gender             | M                           | str | 性別 |
| age                | 29                          | i64 | 年齢 |
| birth              | 1990/6/10                   | str | 誕生日 |
| pref               | 石川県                      | str | 県 |

In [ ]:
customer_master = pl.read_csv("customer_master.csv")
customer_master.head()

In [ ]:
#データ説明の表を作成するためDataFrameを転置する
#print(customer_master.head(1).transpose(include_header = True))

#### 製品マスター

| 列名 | データ例 | データ型 | 列の説明 |
| ---- | ---- | ---- | ---- |
| item_id    | S001     | str | 製品ID |
| item_name  | PC-A     | str | 製品名 |
| item_price | 50000    | i64 | 価格 |

In [ ]:
item_master = pl.read_csv("item_master.csv")
item_master.head()

In [ ]:
#データ説明の表を作成するためDataFrameを転置する
#print(item_master.head(1).transpose(include_header = True))

#### 購入明細データ

| 列名 | データ例 | データ型 | 列の説明 |
| ---- | ---- | ---- | ---- |
| transaction_id | T0000000113         | str | 購入明細ID |
| price          | 210000              | i64 | 購入価格 |
| payment_date   | 2019-02-01 01:36:57 | ste | 購入日時 |
| customer_id    | PL563502            | str | 顧客ID |

In [ ]:
transaction_1 = pl.read_csv("transaction_1.csv")
transaction_1.head()

In [ ]:
#データ説明の表を作成するためDataFrameを転置する
#print(transaction_1.head(1).transpose(include_header = True))

#### 購入明細の詳細データ

| 列名 | データ例 | データ型 | 列の説明 |
| ---- | ---- | ---- | ---- |
| detail_id      | 0           | i64 | 詳細ID |
| transaction_id | T0000000113 | str | 明細ID |
| item_id        | S005        | str | 製品ID |
| quantity       | 1           | i64 | 数量 |

In [ ]:
transaction_detail_1 = pl.read_csv("transaction_detail_1.csv")
transaction_detail_1.head()

In [ ]:
#データ説明の表を作成するためDataFrameを転置する
#print(transaction_detail_1.head(1).transpose(include_header = True))

In [ ]:
# 当然だが、複数の商品を同時に計算しているため、
# 購入明細の金額は Σ(製品価格)(数量) で計算される
transaction_detail_1\
    .pivot(values = "quantity", index = "transaction_id", columns = "item_id", aggregate_function = "count", sort_columns = True, maintain_order = True)\
        .filter( pl.col("transaction_id") == "T0000000231")
#transaction_detail_1[["transaction_id", "item_id"]].unique().sort(by = ["transaction_id"])

### ノック２：データを結合(ユニオン)してみよう

In [ ]:
transaction_2 = pl.read_csv("transaction_2.csv")
transaction_2.head()

In [ ]:
transaction = pl.concat([transaction_1, transaction_2], how = "vertical")
transaction.head()

In [ ]:
print(len(transaction_1))
print(len(transaction_2))
print(len(transaction_1) + len(transaction_2))
print(len(transaction))

In [ ]:
transaction_detail_2 = pl.read_csv("transaction_detail_2.csv")
transaction_detail = pl.concat([transaction_detail_1, transaction_detail_2], how = "vertical")
transaction_detail.head()

### ノック３：売上データ同士を結合(ジョイン)してみよう

In [ ]:
join_data = transaction_detail.join(
    other = transaction[["transaction_id", "payment_date", "customer_id"]],
    on = "transaction_id",
    how = "left")
join_data.head()

In [ ]:
print(len(transaction_detail))
print(len(transaction))
print(len(join_data))

### ノック４：マスタデータを結合(ジョイン)してみよう

In [ ]:
join_data = join_data.join(
    other = customer_master,
    on = "customer_id",
    how = "left")
join_data = join_data.join(
    other = item_master,
    on = "item_id",
    how = "left")
join_data.head()

### ノック5：必要なデータ列を作ろう

In [ ]:
join_data = join_data.with_columns([
    (join_data["quantity"] * join_data["item_price"]).alias("price")
])
join_data[["quantity", "item_price", "price"]].head()

### ノック6：データ検算をしよう

In [ ]:
print(join_data["price"].sum())
print(transaction["price"].sum())

In [ ]:
join_data["price"].sum() == transaction["price"].sum()

### ノック7：各種統計量を把握しよう

In [ ]:
# pandasのjoin_data.isnull.sum()と同じ結果を得る
join_data.null_count()

In [ ]:
#pandas.describe()と同じ結果を得る
import polars.selectors as cs
join_data.select(cs.numeric()).describe()

In [ ]:
print(join_data["payment_date"].min())
print(join_data["payment_date"].max())

### ノック8：月別でデータを集計してみよう

In [ ]:
join_data.dtypes

In [ ]:
join_data = join_data\
    .replace("payment_date",
             join_data["payment_date"].str.strptime(dtype = pl.Datetime, format = "%Y-%m-%d %H:%M:%S"))
join_data = join_data.with_columns([
    pl.col("payment_date").dt.strftime("%Y%m").cast(dtype = pl.Int64).alias("payment_month")
])
join_data[["payment_date", "payment_month"]].head()

In [ ]:
join_data.group_by(by = ["payment_month"]).agg( pl.col("price").sum() ).sort("payment_month")

### ノック9：月別、商品別でデータを集計してみよう

In [ ]:
join_data.group_by(by = ["payment_month", "item_name"]).agg( pl.col("price").sum(), pl.col("quantity").sum() ).sort(["payment_month", "item_name"])

In [ ]:
join_data.pivot(values = ["price", "quantity"], index = "item_name", columns = "payment_month", aggregate_function = "sum", maintain_order = True, sort_columns = True)

### ノック10：商品別の売上推移を可視化してみよう

In [ ]:
import pandas as pd
graph_data = pd.pivot_table(join_data.to_pandas(), index = "payment_month", columns = "item_name", values = "price", aggfunc = "sum")
graph_data.head()

In [ ]:
import matplotlib .pyplot as plt
%matplotlib inline
plt.plot(list(graph_data.index), graph_data["PC-A"], label = "PC-A")
plt.plot(list(graph_data.index), graph_data["PC-B"], label = "PC-B")
plt.plot(list(graph_data.index), graph_data["PC-C"], label = "PC-C")
plt.plot(list(graph_data.index), graph_data["PC-D"], label = "PC-D")
plt.plot(list(graph_data.index), graph_data["PC-E"], label = "PC-E")
plt.legend()